In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代料总表' in file:
        alternate = './' + file

print(f'\talternate: \t{alternate}')

	alternate: 	./替代料总表10-24.xlsx


In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=1)[['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']].fillna('')

df_alternate = df_alternate[df_alternate['替换料1'] != ''].reset_index(drop=True)

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USGS00429LR,USGS00429T,USGS00429,,,,
2,USWL02211H,USWL02211,,,,,
3,USETC35518D,USTC35518,USETC35518,,,,
4,USTHA7040PX,USTHA7040P,,,,,
...,...,...,...,...,...,...,...
44816,USGS01664-2-C,USGS01664-2,,,,,
44817,USASC06160-C,USASC06160,,,,,
44818,USASC16160-C,USASC16160,,,,,
44819,USASC20303-C,USASC20303,,,,,


In [5]:
from tqdm import tqdm

In [6]:
count = 0
while True:
    print('# = = = = = = = ' + str(count) + ' = = = = = = = = = =')
    list_row = []
    for i in tqdm(range(len(df_alternate)), desc='运行中', ncols=77):
        list_sku = list(set(df_alternate.loc[i].tolist()))
        if '' in list_sku:
            list_sku.remove('')

        find = False
        for sku in list_sku:
            for j in range(len(list_row)):
                if sku in list_row[j]:
                    list_row[j] = list(set(list_row[j] + list_sku))
                    find = True
                    break
            if find == True:
                break

        if find == False:
            list_row.append(list_sku)  
        
    if len(list_row) == len(df_alternate):
        break
    else:
        df_alternate = pd.DataFrame(list_row).fillna('')
        df_alternate.columns = ['主料SKU'] + ['替换料'+str(i) for i in range(1, len(df_alternate.columns))]
        count += 1
        print()

# = = = = = = = 0 = = = = = = = = = =


运行中: 100%|█████████████████████████| 44821/44821 [03:57<00:00, 188.62it/s]



# = = = = = = = 1 = = = = = = = = = =


运行中: 100%|█████████████████████████| 44803/44803 [03:57<00:00, 188.48it/s]


In [7]:
import datetime

In [8]:
writer = pd.ExcelWriter('Lennon_整理好的替代料总表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_alternate.to_excel(writer, index=False)
writer.close()